# <font color="darkblue"> Prática 01: Regularização - Heurística Weight Decay </font>

**Objetivos:**


*   Testar a Regressão logística com outra base de dados
*   Implementar a heurística *weight decay* para regularização da função inferida pelo algoritmo de aprendizagem 

**Requisitos de execução:**


*   Upload dos arquivos *logisticregression.py* e *diabetes.csv*

**Atividade 1:**

1. Visitar a base de dados: https://www.kaggle.com/uciml/pima-indians-diabetes-database
2. Carregar os dados do arquivo *diabetes.csv* utilizando o pandas.

    

In [ ]:
import pandas as pd 

diabete = pd.read_csv("diabetes.csv", sep=',')

# Pega o cabecalho do arquivo
print(diabete.head())

**Atividade 2:**

1. Extrair os valores do *DataFrame* pandas e colocar nas variáveis


In [ ]:
Features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',  'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
X = diabete[Features].values
y = diabete.Outcome.values

print(X)
print(y)

**Atividade 3:**

1. Separar os dados em conjunto de treinamento e teste


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x_T, x_test, y_T, y_test = train_test_split(X, y, test_size=0.2, random_state=2698)

sc = StandardScaler()
x_T = sc.fit_transform(x_T)
x_test = sc.fit_transform(x_test)

print("Tamanho treino: " + str(len(x_T)))
print("Tamanho teste: " + str(len(x_test)))

**Atividade 4:**

1. Utilize a classe LogisticRegression, importada do pacote *sklearn.metrics*, para inferir aprendizado dos dados de treinamento;
2. Compute as métricas de aprendizado sobre os dados de teste.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(solver='newton-cg')
model.fit(x_T, y_T)

print(classification_report(y_test, model.predict(x_test)))

**Atividade 5:**

1. Utilize a classe LogisticRegression, importada do arquivo *logisticregression.py*, para inferir aprendizado dos dados de treinamento;
2. Compute o $E_{in}$ sobre os dados de treino e o $E_{out}$ sobre os dados de teste;
2. Compute as métricas de aprendizado sobre os dados de teste.

In [ ]:
from sklearn.metrics import classification_report
from logisticregression import LogisticRegression

lrY = [+1 if value == 1 else -1 for value in y_T]
lr_y_test = [+1 if value == 1 else -1 for value in y_test]

#Regressão Logistica
classifier = LogisticRegression(0.1, 1000, 32)
classifier.fit(x_T, lrY)

def error(pred, y):
    N = len(y)
    error = 0
    for i in range(N):
        if(pred[i] != y[i]):
          error += 1
    error /= N
    return error

#Computando o erro dentro da amostra (Ein)
pred = classifier.predict(x_T)
print("Ein = " + str(error(pred, lrY)))

#Computando o erro fora da amostra (Eout)
pred = classifier.predict(x_test)
print("Eout = " + str(error(pred, lr_y_test)))

#Métricas de aprendizado
pred = classifier.predict(x_test)
print(classification_report(lr_y_test, pred))

Atividade 6:

1. Implemente a heurística *WeightDecay* utilizando a penalidade $L_2$

In [46]:
from logisticregression import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

import warnings
warnings.filterwarnings("error")

class WeightDecay:
   
    # Infere o vetor w da funçao hipotese
    #Executa a heuristica Weigth Decay
    def fit(self, _X, _y):
        x_train, x_val, y_train, y_val = train_test_split(_X, _y, test_size=0.2, random_state=2698)
        
        d = x_train.shape[1]
        N = x_train.shape[0]
        w = np.zeros(d, dtype=np.float128)
        
        lambdas = sorted([10 ** x for x in range(-5, 5)])
        print(lambdas)
        lambdas.append(0)
        bestEout = 100
 
        for lam in lambdas:
            classifier = LogisticRegression(0.1, 1000, 32, lam)
            try:
              classifier.fit(x_train, y_train)
            except RuntimeWarning:
              continue 
            #Computando o erro quadrático (Eout)
            eOut = self.getEout(classifier.predict(x_val), y_val)                
            
            print("LA: " + str(classifier.lam) + "  Eval = " + str(eOut))
            if eOut < bestEout:
                bestEout = eOut
                self.classifier = classifier
                    
        self.w = self.classifier.w
        #print("Best Lamb: " + str(self.classifier.lam))
        
    #funcao hipotese inferida pela regressa logistica  
    def predict_prob(self, X):
        return [(1 / (1 + np.exp(-(self.w[0] + self.w[1:].T @ x)))) for x in X]

    #Predicao por classificação linear
    def predict(self, X):
        return [1 if (1 / (1 + np.exp(-(self.w[0] + self.w[1:].T @ x)))) >= 0.5 
                else -1 for x in X]

    def getW(self):
        return self.w

    def getRegressionY(self, regressionX, shift=0):
        return (-self.w[0]+shift - self.w[1]*regressionX) / self.w[2]
    
    def getEout(self, pred, y_val):
        #Computando o erro quadrático (Eout)
        N_val = len(y_val)
        eOut = 0
        for i in range(N_val):
            if(pred[i] != y_val[i]):
              eOut += 1 
        eOut /= N_val
        return eOut

**Atividade 7:**

1. Utilize a classe WeightDecay para inferir aprendizado dos dados de treinamento;
2. Compute o $E_{in}$ sobre os dados de treino e o $E_{out}$ sobre os dados de teste;
3. Compute as métricas de aprendizado sobre os dados de teste e compare os resutlados.

In [ ]:
from sklearn.metrics import classification_report

#Weigth Decay heuristic
classifier = WeightDecay()
classifier.fit(x_T, lrY)

#Computando o erro dentro da amostra (Ein)
pred = classifier.predict(x_T)
print("\nEin = " + str(error(pred, lrY)))

#Computando o erro dentro da amostra (Eout)
pred = classifier.predict(x_test)
print("Eout = " + str(error(pred, lr_y_test)))

#Métricas de aprendizado
pred = classifier.predict(x_test)
print(classification_report(lr_y_test, pred))

Atividade 8:

1. Implementar a validação da Logistic Regression através da classe *GridSearchCV* do pacote *sklearn.model_selection*;
2. Imprimir os parâmetros do melhor classificador;
3. Imprimir os $E_{in}$ e $E_{out}$  e as métricas de aprendizado.

Parâmetros:


*   *estimator* : instância do classificador cujos hiperparâmetros serão analisados;
*   *cv* : número de divisões do conjunto de treinamento para ser usado na técnica de validação cruzada (10 é um bom valor observado na prática);
*   *param_grid* : conjunto de parâmetros a serem combinados durante a fase de validação.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

lr = LogisticRegression(solver='newton-cg')

# Create the random grid
param_grid = {
              'C' : np.logspace(-5, 5, 10),
              }

CV_rf = GridSearchCV(estimator=lr, param_grid=param_grid, cv = 10, verbose=2, n_jobs=-1)

# Fit the random search model
CV_rf.fit(x_T, y_T)

print(CV_rf.best_estimator_)


print('Ein: %0.4f' % (1 - accuracy_score(y_T, CV_rf.predict(x_T))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, CV_rf.predict(x_test))))
print(classification_report(y_test, CV_rf.predict(x_test)))